<a href="https://colab.research.google.com/github/xchen35/Model_Imagenet32/blob/master/AutoKeras_Experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install autokeras

In [2]:
from __future__ import print_function
import keras
import numpy as np
import os
from autokeras import ImageClassifier

Using TensorFlow backend.


In [3]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pickle
def unpickle(file):
    with open(file, 'rb') as fo:
        data = pickle.load(fo, encoding='bytes')
    return data

In [0]:
def load_data(data_dir, negatives=False):
    """
    Return train_data, train_labels, test_data, test_labels
    """

    # training data
    train_data = None
    train_labels = []

    # train_data_dict

    for i in range(1, 11):
        train_data_dict = unpickle(data_dir + "/train_data_batch_{}".format(i))
        if i == 1:
            train_data = train_data_dict['data']
        else:
            train_data = np.vstack((train_data, train_data_dict['data']))
#         train_filenames += train_data_dict[b'filenames']
        train_labels += train_data_dict['labels']

    train_data = train_data.reshape((len(train_data), 3, 32, 32))
    print(len(train_data))
    if negatives:
        train_data = cifar_train_data.transpose(0, 2, 3, 1).astype(np.float32)
    else:
        train_data = np.rollaxis(train_data, 1, 4)
#     train_filenames = np.array(train_filenames)
    train_labels = np.array(train_labels)

    # test data
    # cifar_test_data_dict
    # 'batch_label': 'testing batch 1 of 1'
    # 'data': ndarray
    # 'labels': list

    test_data_dict = unpickle(data_dir + "/val_data")
    test_data = test_data_dict['data']
    test_labels = test_data_dict['labels']

    test_data = test_data.reshape((len(test_data), 3, 32, 32))
    if negatives:
        test_data = test_data.transpose(0, 2, 3, 1).astype(np.float32)
    else:
        test_data = np.rollaxis(test_data, 1, 4)
    test_labels = np.array(test_labels)

    return train_data, train_labels, \
           test_data, test_labels

In [6]:
# The data, split between train and test sets:
x_train, y_train, x_test, y_test = \
        load_data('/content/drive/My Drive/out_data_train')
# print('x_train shape:', x_train.shape)
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)

100000
x_train shape: (100000, 32, 32, 3)
y_train shape: (100000,)
x_test shape: (2009, 32, 32, 3)
y_test shape: (2009,)


In [7]:
y_train = y_train.reshape(len(y_train),1)
y_test = y_test.reshape(len(y_test),1)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

y_train shape: (100000, 1)
y_test shape: (2009, 1)


In [8]:
clf = ImageClassifier(verbose=True, augment=True, searcher_args={'trainer_args':{'max_iter_num':3}})
# clf.fit(x_train, y_train, time_limit=(1*60*60))
clf.fit(x_train, y_train, time_limit=(1*60*20))

Saving Directory: /tmp/autokeras_3V0QLI
Preprocessing the images.
Preprocessing finished.

Initializing search.
Initialization finished.


+----------------------------------------------+
|               Training model 0               |
+----------------------------------------------+
                                                                                                    
Saving model.
+--------------------------------------------------------------------------+
|        Model ID        |          Loss          |      Metric Value      |
+--------------------------------------------------------------------------+
|           0            |   17.872286478678387   |  0.07066666666666667   |
+--------------------------------------------------------------------------+


+----------------------------------------------+
|               Training model 1               |
+----------------------------------------------+
Epoch-3, Current Metric - 0.128:  10%|██▌                      | 

In [9]:
clf.final_fit(x_train, y_train, x_test, y_test, retrain=False)

Epoch-21, Current Metric - 0.0054753608760577405:   4%|▎       | 30/782 [00:02<00:50, 14.88 batch/s]

KeyboardInterrupt: ignored

In [10]:
from sklearn.metrics import accuracy_score
y_prediction = clf.predict(x_test)
accuracy_score(y_true=y_test, y_pred=y_prediction)

0.003484320557491289